# Case study: queue analysis

This case study illustrates the use of the staircase package for queue analysis.  In this example we have a number of vessels (i.e. ships) which arrive offshore and await their turn to enter a harbour where they will be loaded with cargo.  We will examine the queue, which is composed of all vessels which are offshore but yet to enter the harbour, for the year 2020.

The data used is this case study is synthetic and fictional.  Both data and the notebook for this tutorial can be obtained from the [github site](https://github.com/venaturum/staircase/tree/master/docs/examples).

In [ ]:
import pandas as pd
import staircase as sc
import matplotlib.pyplot as plt

We begin by importing the queue data into a pandas.DataFrame instance. Each row corresponds to a vessel. The first column gives the time at which the vessel arrives offshore (enters the queue), and the second column gives the time at which the vessel enters the harbour (leaves the queue).  A [NaT](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#datetimes) value in either of these columns indicates the vessel entered the queue prior to 2020, or left the queue after 2020, however this approach does not require these values to be NaT.  The third column gives the weight of cargo destined for the vessel.  Note, for the staircase approach to work we require every vessel, that was in the queue at some point in 2020, to appear in the dataframe.

In [ ]:
data = pd.read_csv(r"data/vessel_queue.csv", parse_dates=['enter', 'leave'], dayfirst=True)
data

The layer method can be used with array-like parameters.  The creation of a step function to quantify the size of the queue is as simple as calling the layer method with a vector of times that vessels enter the queue, and a vector of times that vessels leave the queue - the columns "enter" and "leave" respectively:

In [ ]:
queue = sc.Stairs(use_dates=True).layer(data.enter, data.leave)
queue.plot()

Assuming that no vessels arrive precisely at midnight on the 1st of Jan, we expect the number of vessels in the queue at this time to be equal to the number of NaT values in the "enter" column:

In [ ]:
queue(pd.Timestamp('2020-01-01'))

In [ ]:
data.enter.isna().sum()

The Stairs class comes with a hist function (histogram) which can be pretty useful for understanding the distribution of the values of the corresponding step function.  The result is returned as a pandas.Series, indexed by a pandas.IntervalIndex.  If we don't supply the bins as a parameter it will use unit bins which cover the step function's range:

In [ ]:
queue_hist = queue.hist(pd.Timestamp('2020'), pd.Timestamp('2021'))
queue_hist

We know the queue length has to be a whole number so we can change the pandas.IntervalIndex accordingly before plotting

In [ ]:
queue_hist.index = queue_hist.index.left
ax = queue_hist.plot.bar()
ax.set_xlabel('queue size', fontsize='12')
ax.set_ylabel('frequency', fontsize='12')
;

Another useful queue metric is the "queue tonnes".  This is the sum of the cargo tonnes destined for vessels in the queue.  A step function representing this variable is also straightforward by using the third parameter of the layer method - the values representing how much the step function should increase or decrease whenever the corresponding vessels enter or leave the queue:

In [ ]:
queue_tonnes = sc.Stairs(use_dates=True).layer(data.enter, data.leave, data.tonnes)
queue_tonnes.plot()

We can use this queue_tonnes object to answer questions like "what was the maximum queue tonnes in 2020?"

In [ ]:
queue_tonnes.max()

.. or "what was the average size of the queue in 2020?"

In [ ]:
queue_tonnes.mean(pd.Timestamp('2020'), pd.Timestamp('2021'))

or "what fraction of the year was the queue_tonnes larger than 1,500,000 tonnes?"

In [ ]:
(queue_tonnes > 1500000).mean(pd.Timestamp('2020'), pd.Timestamp('2021'))

.. or "what was the median size of the queue in March 2020?"

In [ ]:
queue_tonnes.median(pd.Timestamp('2020-3-1'), pd.Timestamp('2020-4-1'))

The median gives us the 50th percentile, but we might be interested in the 80th percentile?  We can do that:

In [ ]:
queue_tonnes.percentile(80, pd.Timestamp('2020-3-1'), pd.Timestamp('2020-4-1'))

In fact we can even get a percentile function, represented by a Stairs object itself.

In [ ]:
percentile = queue_tonnes.percentile_stairs(pd.Timestamp('2020'), pd.Timestamp('2021'))
percentile

We can plot this function of course, since it is represented by a Stairs object:

In [ ]:
percentile.plot()

The 100th percentile should be the same as the maximum queue tonnes we found earlier.  Let's check:

In [ ]:
percentile(100) == queue_tonnes.max()

What is the 40th, 65th, 77th and 90th percentiles?  The sample method, which is aliased by \_\_call\_\_, can be called with a vector of values at which to evaluate the step function too.

In [ ]:
percentile([40, 65, 77, 90])

The percentile function is essentially the inverse of an [empirical cumulative distribution function](https://en.wikipedia.org/wiki/Empirical_distribution_function).  Perhaps, unsurprisingly we can generate an ecdf like so:

In [ ]:
ecdf = queue_tonnes.ecdf_stairs(pd.Timestamp('2020'), pd.Timestamp('2021'))
ecdf

In [ ]:
ecdf.plot()

Earlier we found that the queue tonnes were strictly above 150,000t about 9.87% of the time.  The ecdf can tell us what fraction of the time the queue tonnes was less than or equal to 150,000t.  (We'd expect these results to add to 1).

In [ ]:
ecdf(1500000)

We can also then discover the fraction of time that 100,000 < queue tonnes <= 150,000 like so:

In [ ]:
ecdf(1500000) - ecdf(100000)

This sort of information is essentially a single bin in histogram data.  Earlier we used the Stairs.hist function on the queue length, with default bin sizes.  Since the range of the queue tonnes values is so large we will provide our own bins (defined by specifiying the edges) to generate a histogram for the queue tonnes.

In [ ]:
queue_tonnes_hist = queue_tonnes.hist(pd.Timestamp('2020'), pd.Timestamp('2021'), bin_edges = range(0, 2100000, 100000))
queue_tonnes_hist

As we did earlier for the queue length histogram, we can quickly create a chart with pandas Series plotting functions.

In [ ]:
ax = queue_tonnes_hist.plot.bar()
ax.set_xlabel('queue size in tonnes', fontsize='12')
ax.set_ylabel('frequency', fontsize='12')
;

Returning to our queue plots... They're pretty noisy. We can reduce some of this noise by using the rolling_mean method belonging to the Stairs class.  The method uses the fact that a rolling mean of a piecewise constant function, i.e. a step function, will be a piecewise linear function.  And a piecewise linear function can be described by the (x,y) coordinates at each end of the "pieces".  The result is returned as a pandas.Series.

In this example we apply a rolling mean with a 7 day, centred, window around every point in the step function.  We do this by specifying the window with a tuple, detailing the distances from the point to the boundaries of thw window - in this case, 3.5 days either side of the point.

In [ ]:
queue.rolling_mean(window=(-pd.Timedelta(3.5, 'D'), pd.Timedelta(3.5, 'D')))

Note that our original data was contained to the year 2020 but the data in our rolling mean has spilled over into 2019, and 2020.  This is because our step function theoretically extends to -infinity and +infinity.  In this case this effect is unwanted.  Similar to the application of rolling means with pandas.Series, we will actually have to sacrifice some of our "usable domain" to create the rolling mean. We do this by specifying a domain of our step function when applying the rolling mean:

In [ ]:
queue.rolling_mean(
    window=(-pd.Timedelta(3.5, 'D'), pd.Timedelta(3.5, 'D')),
    lower=pd.Timestamp('2020'),
    upper=pd.Timestamp('2021'),
)

Now we can see that we only have dates in our rolling mean data where the window entirely fits within the domain we specified.  The real use of this data is realised when plotting, and we can leverage the [plot function of pandas.Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.plot.html) to this effect.

In [ ]:
queue.rolling_mean(
    window=(-pd.Timedelta(3.5, 'D'), pd.Timedelta(3.5, 'D')),
    lower=pd.Timestamp('2020'),
    upper=pd.Timestamp('2021'),
).plot()

We can see that applying the rolling window has removed the noise and allowed us to look at trend.  We can get an ever coarser trend by using a larger window, for example 30 day centred window results in the following graph.

In [ ]:
queue.rolling_mean(
    window=(-pd.Timedelta(15, 'D'), pd.Timedelta(15, 'D')),
    lower=pd.Timestamp('2020'),
    upper=pd.Timestamp('2021'),
).plot()

Another method to remove noise is to apply a static mean, rather than a rolling mean.  For example, we could use calculate a daily mean.  To achieve this let's use Python's zip function, list comprehension and a pandas.Series to derive and collect this data:

In [ ]:
yr2020 = pd.date_range('2020', '2021')
daily_mean_queue = pd.Series(
    [queue.mean(d1,d2) for d1,d2 in zip(yr2020[:-1], yr2020[1:])],
    index = yr2020[:-1]
)
daily_mean_queue

A new way to achieve the exact same result, as of v1.5.0, is to use the sample method.  The sample method was used earlier to calculate the 40th, 65th, 77th and 90th percentiles for queue_tonnes, however we accessed it via its alias \_\_call\_\_ which allows us to simply write () instead of sample(). The sample method has been extended to be able to specify an aggregating function, and a window, similar to the rolling mean.  If we apply the mean function, and a 1 day leading window, then we get exactly the same result as the cell above (and it runs slightly faster too). 

In [ ]:
pd.Series(
    queue.sample(yr2020[:-1], aggfunc='mean', window=(pd.Timedelta(0), pd.Timedelta(1, 'D'))),
    index = yr2020[:-1],
)

As with the rolling mean, this data is best interpreted with a plot:

In [ ]:
daily_mean_queue.plot()

This time we can use [pandas.Series.rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.rolling.html) to get a rolling mean - although this will be a rolling mean of daily averages.  This data can of course be plotted with [matplotlib](https://matplotlib.org/) or [seaborn](https://seaborn.pydata.org/), but for now let's keep leveraging the pandas.Series plotting methods:

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
daily_mean_queue.plot(ax=ax, label="queue size")
daily_mean_queue.rolling(7, center=True).mean().plot(ax=ax, linewidth=3, label="rolling mean")
ax.legend()